# Erste Analysen und Visualisierungen erstellen

In [ ]:
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from nltk.util import ngrams
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
import numpy as np
from wordcloud import WordCloud
import plotly.express as px
import re

In [ ]:
df = pd.read_hdf("pdf_texts.h5", key="df_texts")
df

## 1. Häufigkeitsanalysen 

### 1.1. Worthäufigkeiten übergreifend (alle PDFs)

In [ ]:
# Importieren der zu entfernenden Stopwörter
nltk.download('stopwords')
stopger = stopwords.words('german')
nltk.download('punkt')

# Hinzufügen weiterer Stopwörter
newStopwords = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'vgl', '\x97', '•', '■', 'v',
                'beim', 'de','—','ge','la','be','en','que','el','ten','ver','gen','sei','nen','del','nen', 'se','schen','un','land','te','ei','aires',
                'las', 'los', '«']

#Erweitern der heruntergeladenen Stopwörter mit den selbst definierten "newStopwords":
stopger.extend(newStopwords)

##### Schrittweise Herleitung der einzelnen Arbeitsschritte:

In [ ]:
# Alle Inhalte aus der Spalte "text" des Dataframes in eine Liste überführen
content = df['text'].tolist()

# Optional: Konvertierung aller Wörter in Kleinschreibung: 
content = [text.lower() for text in content]  

print(content[0])

In [ ]:
# Entfernen von Satzzeichen und Tokenisierung
listofthings = []
for entry in content:
    for c in string.punctuation:
        entry = entry.replace(c, " ")
    words = entry.split()
    listofthings.extend(words)

In [ ]:
print(listofthings)

In [ ]:
# Entfernen von Zahlen (bspw. Seitenzahlen, andere Zahlen)
words = [word for word in listofthings if not word.isdecimal()]
print(words)

In [ ]:
#print(stopger)

In [ ]:
# Entfernen der Stopwörter aus der Gesamtwortliste: 
tokens_without_sw = [word for word in words if word not in stopger]
print(tokens_without_sw)

In [ ]:
# Zählen der Häufigkeit der verbleibenden Wörter
counts = Counter(tokens_without_sw)

# Top 20 häufigste Wörter ausgeben
top_20_words = counts.most_common(20)
top_150_words = counts.most_common(150)
print(top_20_words)

Zusammenfassung aller Schritte:

In [ ]:
# Alle Inhalte aus der Spalte "text" des Dataframes in eine Liste überführen
content = df['text'].tolist()
# Optional: Konvertierung aller Wörter in Kleinschreibung: 
content = [text.lower() for text in content]  

# Manuelle Tokenisierung und Entfernung von Satzzeichen: 
listofthings = []
for entry in content:
    for c in string.punctuation:
        entry = entry.replace(c, " ")
    words = entry.split()
    listofthings.extend(words)

# Entfernen von Zahlen (bspw. Seitenzahlen, andere Zahlen)
words = [word for word in listofthings if not word.isdecimal()]

# Entfernen der Stopwörter aus der Gesamtwortliste: 
tokens_without_sw = [word for word in words if word not in stopger]

# Zählen der Häufigkeit der verbleibenden Wörter
counts = Counter(tokens_without_sw)

# Top 20 häufigste Wörter ausgeben
top_20_words = counts.most_common(20)
top_150_words = counts.most_common(150)
print(top_20_words)

#### Funktion

Clean Text: 

In [ ]:
def clean_and_tokenize(df, stopger):
  # Funktion zum Bereinigen eines einzelnen Textes
    def process_text(text):
        # Konvertiere Text in Kleinbuchstaben
        text = text.lower()
        
        # Entferne Satzzeichen
        for c in string.punctuation:
            text = text.replace(c, " ")
        
        # Tokenisiere den Text
        words = text.split()
        
        # Entferne Zahlen
        words = [word for word in words if not word.isdecimal()]
        
        # Entferne Stopwörter
        cleaned_tokens = [word for word in words if word not in stopger]
        return cleaned_tokens
    
    # Wende die Funktion auf jede Zeile der 'text'-Spalte an und speichere das Ergebnis in 'cleaned_tokens'
    df['cleaned_tokens'] = df['text'].apply(process_text)

    return df


In [ ]:
df = clean_and_tokenize(df, stopger)
df

In [ ]:
def get_frequency(df): 

    # Alle Inhalte aus der Spalte "text" des Dataframes in eine Liste überführen
    words = df['cleaned_tokens'].tolist()
    content = []
    for wordlist in words:
        for word in wordlist: 
            content.append(word)

    # Zählen der Häufigkeit der verbleibenden Wörter
    counts = Counter(content)
    
    # Top 20 häufigste Wörter ausgeben
    top_20_words = counts.most_common(20)
    top_150_words = counts.most_common(150)

    #frequencies = {"top 20": top_20_words, "top_150": top_150_words}
    #return frequencies
    return top_20_words, top_150_words
        

In [ ]:
result = get_frequency(df)

Da die Funktion 2 Variablen zurückgibt, werden diese hintereinander als Variable in die Liste "result" gespeichert, d.h. als erstes Element der Liste wird die Variable `top_20_words` und als zweites Listenelement die Variable `top_150_words` abgelegt. Entsprechend können über die Listenindizes die Variablen angesprochen werden: 

- result[0] enthält also die Variable `top_20_words`
- result[1] enthält die Variable `top_150_words`

Analog können weitere Häufigkeiten in der Funktion hinzugefügt und angesprochen werden. 

In [ ]:
print(result[0])

### 1.2. Visualisierungen

#### 1.2.1. Kreisdiagramm

In [ ]:
# Visualisiere das Kreisdiagramm mit Plotly Express
def create_plotly_circle(word_count):
    labels = [word[0] for word in word_count]
    sizes = [word[1] for word in word_count]
        
    fig = px.pie(word_count, names=labels, values=sizes, title='Top Wörter', height=500)
    fig.show()

# Kreisdiagramm erstellen
create_plotly_circle(result[0])

#### 1.2.2. Balkendiagramm

In [ ]:
# Funktion zum Erstellen eines Balkendiagramms
def create_bar_chart(word_count):
    labels = [word[0] for word in word_count]
    sizes = [word[1] for word in word_count]
    
    fig = px.bar(x = labels, y = sizes, height=500)
    return fig

# Balkendiagramm erstellen
fig2 = create_bar_chart(result[0])
fig2.show()

In [ ]:
# Diagramm als Bild speichern:
# fig2.savefig('plot.jpg', format='jpg', dpi=300)  # Als JPG speichern
# fig2.savefig('plot.png', format='png', dpi=300)  # Als PNG speichern

#### 1.2.3. Wordcloud

In [ ]:
# Erstellen einer Wörterbuch-Datenstruktur für die WordCloud
word_freq_dict = dict(result[1])

# Erstellen und Anzeigen der WordCloud
wc = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_freq_dict)

plt.figure(figsize=(10, 5))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')  # Achsen ausblenden
plt.show()

In [ ]:
# Speichern: 
# wc.to_file('wordcloud.png')  # Speichern als PNG-Datei

## 2.1. N-Gram

In [ ]:
# N-Gramm-Erstellung (z.B. für 2-Gramme)
def generate_ngrams(tokens, n=2):
    return list(ngrams(tokens, n))

In [ ]:
# Berechne die N-Gramme für alle Texte
n = 2  # Für bigrams (2-Gramme), ändere dies für trigramme (n=3) etc.
df['ngrams'] = df['cleaned_tokens'].apply(lambda x: generate_ngrams(x, n))

In [ ]:
# Alle N-Gramme aus allen Texten kombinieren
all_ngrams = [ngram for sublist in df['ngrams'] for ngram in sublist]

# Zähle die Häufigkeit der N-Gramme
ngram_counts = Counter(all_ngrams)

# Wähle die häufigsten 10 N-Gramme aus
top_ngrams = ngram_counts.most_common(20)

In [ ]:
# Erstelle eine DataFrame für die Top-N-Gramme zur besseren Visualisierung
ngram_df = pd.DataFrame(top_ngrams, columns=['ngram', 'count'])
ngram_df['ngram'] = ngram_df['ngram'].apply(lambda x: ' '.join(x))  # Umwandeln von Tupeln in Strings

In [ ]:
# Visualisierung der häufigsten N-Gramme mit Plotly Express
ngram_df = ngram_df.sort_values(by='count', ascending=True)
fig = px.bar(ngram_df, x='count', y='ngram', orientation='h', title=f'Top {n}-Gramme',  labels={'count': 'Häufigkeit', 'ngram': f'{n}-Gramme'},  
             color='count',  color_continuous_scale='Viridis',  height=600)

fig.show()